In [ ]:
# GENERATING PRELIMINARY PLOTS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#complete methodology table as dataframe
ms_complete = pd.read_csv('methods.csv')

In [3]:
#filter to include only columns 'Tools', 'Year', 'PIM', drop unfilled rows, convert year to int values
ms_filtered = ms_complete[["Tools", "Year", "PIM "]].dropna().reset_index()
ms_filtered.Year = ms_filtered.Year.astype(int)

In [ ]:
# BARCHART

In [15]:
#filter by PIM technology (PNM / PUM) or year (2013 - 2024)
focus = ['PUM']
col = 'PIM '

mask = ms_filtered[col].isin(focus)
ms_focused = ms_filtered[mask]

#switch to look at different table
current_table = ms_focused

In [16]:
#get names and counts of unique tools
a = current_table.Tools.str.split(', ', expand=True).to_numpy()
all_tools, all_counts = np.unique(a[a != np.array(None)], return_counts=True)

In [ ]:
#plot all tools
plt.title("All Tools All Occurences")
plt.bar(all_tools, all_counts)
plt.xticks(rotation = -90)
plt.show()

In [ ]:
#plot tools with greater than one occurence
to_include = []
tools = []
counts = []
for i in range(len(all_counts)):
    if all_counts[i] > 1:
        tools.append(all_tools[i])
        counts.append(all_counts[i])
plt.bar(tools, counts)
plt.title(f"{focus[0].strip('[]')} Tools Occurrences > 1") #adjust
plt.xticks(rotation = -90)
plt.show()

In [ ]:
# STACKED BARCHART

In [196]:
#FUNCTION DEFINITIONS
def focus_count(col, focus, tools):
    #create numpy array of tools used
    mask = ms_filtered[col].isin(focus)
    ms_focused = ms_filtered[mask]
    a = ms_focused.Tools.str.split(', ', expand=True).to_numpy()

    #return list of counts
    focus_counts = []
    for t in tools:
        focus_counts.append((a == t).sum())
    return focus_counts

In [197]:
#get all tools with count greater than one (list: tools)
a = ms_filtered.Tools.str.split(', ', expand=True).to_numpy()
all_tools, all_counts = np.unique(a[a != np.array(None)], return_counts=True)
to_include = []
tools = []
for i in range(len(all_counts)):
    if all_counts[i] > 1:
        tools.append(all_tools[i])

In [ ]:
#get counts for each tool separated by PIM version
tool_counts = {
    'PNM': focus_count('PIM ', ['PNM'], tools),
    'PUM': focus_count('PIM ', ['PUM'], tools),
}

In [199]:
#alternatively: counts separated by year
tool_counts = {str(k):None for k in range(2013, 2025)}
for y in tool_counts:
    tool_counts[y] = focus_count('Year', [int(y)], tools)

In [202]:
#or years grouped by four
tool_counts = {
    '2013-2016': focus_count('Year', list(range(2013, 2017)), tools),
    '2017-2020': focus_count('Year', list(range(2017, 2021)), tools),
    '2021-2024': focus_count('Year', list(range(2021, 2025)), tools),
}

In [ ]:
#chart
f, ax = plt.subplots()
bottom = np.zeros(len(tools))
for tool, count in tool_counts.items():
    p = ax.bar(tools, count, label=tool, bottom=bottom)
    bottom += count
ax.set_title("Tools Occurences > 1")
ax.legend()
plt.xticks(rotation = -90)
plt.show()